# Data analysis of Hes1 oscillations in single cells

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [5]:
folder_path = 'single-cell-su'

# Get a list of all files in the folder
all_files = os.listdir(folder_path)

# Filter the list to get only CSV files
csv_files = [file for file in all_files if file.endswith('.csv')]

# Initialize an empty list to hold DataFrames
dfs = []

# Iterate over each CSV file, read it, and add the 'source_file' column
for file in csv_files:
    file_path = os.path.join(folder_path, file)  # Get the full path of the file
    df = pd.read_csv(file_path, header=[0, 1], encoding='ISO-8859-1')
    
    # Drop the first row (it contains the units)
    df = df.drop(index=0)

    # replace 'Unnamed' entries in column names with empty strings and get correct head names
    df.columns = pd.MultiIndex.from_tuples(
    [(col[0] if 'Unnamed' not in col[0] else '',
      col[1] if 'Unnamed' not in col[1] else '') for col in df.columns])
    df.columns = [' '.join(col).strip() for col in df.columns.values]

    # Add a new column with the filename (without the directory)
    df['source_file'] = file

    # Add a new column with the DAPT dosage
    if '_0_su' in file:
      df['SU dosage'] = 0
    elif '_2_su' in file:
       df['SU dosage'] = 2
    elif '_10_su' in file:
       df['SU dosage'] = 10
    elif '_20_dapt' in file:
       df['SU dosage'] = 20
    
    # change column types
    df = df.apply(pd.to_numeric, errors='ignore') # Convert all columns to numeric  
    df['Spot track ID'] = df['Spot track ID'].astype(int)
    df['Label'] = df['Label'].astype(str)

    # adding a time column
    df['Time'] = df['Spot frame']*10
    
    # sort the files in order of cell number and time
    df = df.sort_values(by=['Spot track ID', 'Time'], ascending=True)
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dfs, ignore_index=True)

# adding a time column
#merged_df['Time'] = merged_df['Spot frame']*10

# # sort data by Spot track ID
# merged_df = merged_df.sort_values(by=['Spot track ID', 'Time'], ascending=True)


# Generate a unique numerical index for each combination of 'source_file' and 'Spot track ID'
merged_df['Cell ID'] = merged_df.groupby(['source_file', 'Spot track ID']).ngroup()

merged_df

# Save the DataFrame to a CSV file
merged_df.to_csv('all_su_data.csv')

In [4]:
merged_df


,Label,ID,Spot N links,Spot center intensity Center ch1,Spot center intensity Center ch2,Spot center intensity Center ch3,Spot frame,Spot intensity Mean ch1,Spot intensity Std ch1,Spot intensity Min ch1,...,Spot quick mean Mean ch1,Spot quick mean Mean ch2,Spot quick mean Mean ch3,Spot radius,Spot track ID,Track N spots,source_file,DAPT dosage,Time,Cell ID
0,NT,0.0,1.0,227.434075,713.906950,518.718520,0.0,227.804878,112.907964,0.0,...,213.461538,508.192308,512.884615,5.0,0,120.0,230707_0_dapt_001_table-Spot.csv,0,0.0,0
1,NT,1.0,2.0,262.702183,926.069979,535.693465,1.0,215.053763,118.968608,0.0,...,205.666667,495.200000,527.433333,5.0,0,120.0,230707_0_dapt_001_table-Spot.csv,0,10.0,0
2,NT,2.0,2.0,292.320513,797.651872,528.349539,2.0,278.095238,123.858413,40.0,...,265.000000,511.928571,527.142857,5.0,0,120.0,230707_0_dapt_001_table-Spot.csv,0,20.0,0
3,NT,3.0,2.0,266.795222,673.049660,503.538233,3.0,257.349398,112.304946,0.0,...,244.444444,423.222222,506.481481,5.0,0,120.0,230707_0_dapt_001_table-Spot.csv,0,30.0,0
4,NT,4.0,2.0,241.802911,752.240132,543.007203,4.0,221.428571,117.228465,0.0,...,206.666667,490.962963,546.629630,5.0,0,120.0,230707_0_dapt_001_table-Spot.csv,0,40.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18437,SM,367.0,2.0,85.206673,1108.207642,386.855412,117.0,78.012422,55.821365,0.0,...,76.530612,742.612245,392.571429,5.0,2,122.0,230818_2_dapt_006_table-Spot.csv,2,1170.0,156
18438,SM,368.0,2.0,85.657694,934.519073,392.491940,118.0,78.757764,58.038325,0.0,...,71.000000,609.720000,389.720000,5.0,2,122.0,230818_2_dapt_006_table-Spot.csv,2,1180.0,156
18439,SM,369.0,2.0,86.616514,971.542399,386.352229,119.0,78.940397,58.959334,0.0,...,81.086957,779.934783,385.130435,5.0,2,122.0,230818_2_dapt_006_table-Spot.csv,2,1190.0,156
18440,SM,370.0,2.0,74.345900,849.132521,422.004659,120.0,71.847134,57.421314,0.0,...,72.600000,475.560000,411.640000,5.0,2,122.0,230818_2_dapt_006_table-Spot.csv,2,1200.0,156
